In [ ]:
# !pip install pystan==2.19.1.1
# !pip install fbprophet
# !pip install plotly

# data path in VM local file system: home/data/in/ggtrends_downloads, home/data/in/weather_downloads
# data path in HDFS: hdfs://cluster-bda4-m/user/root/project/data/in/ggtrends, hdfs://cluster-bda4-m/user/root/project/data/in/weather

# To upload data from VM local file system to HDFS execute command similar to the following one:
# hdfs dfs -copyFromLocal home/data/in/ggtrends_downloads/Instagram_Hamburg_historic.csv hdfs://cluster-bda4-m/user/root/project/data/in/ggtrends

In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.types import FloatType

from fbprophet import Prophet

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

In [ ]:
def symmetric_mean_absolute_percentage_error(A, F):
    with np.errstate(divide='ignore', invalid='ignore'):
        tmp = 2 * np.abs(F-A) / (np.abs(A) + np.abs(F))
    tmp[np.isnan(tmp)] = 0
    return np.sum(tmp) / len(tmp) * 100

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Time series data analysis with Spark") \
    .config("spark.redis.ssl", "true") \
    .getOrCreate()

In [ ]:
weather_df = spark.read.csv("hdfs://cluster-bda4-m/user/root/project/data/in/weather/merged_historic_Hamburg.csv", inferSchema="true", header="true")
website_df = spark.read.csv("hdfs://cluster-bda4-m/user/root/project/data/in/ggtrends/Instagram_Hamburg_historic.csv", inferSchema="true", header="true")


In [ ]:
weather_df

In [ ]:
# weather_df = weather_df.withColumnRenamed("Date time", "date")
# weather_df = weather_df.withColumn("date", F.to_timestamp(F.col("date"),"MM/dd/yyyy HH:mm:ss").alias("date"))

# website_df = website_df.select(F.to_timestamp(F.col("date"),"MM-dd-yyyy").alias("date"), F.col("value").alias("y"))


# columns = [
#     "date",
#     "Address",
#     "Temperature",
#     "Relative Humidity",
#     "Wind Speed",
#     "Visibility",
#     "Cloud Cover",
#     "y"]



# df = weather_df.join(website_df, how="inner", on="date").select(*columns)
# partition = Window.partitionBy("Address").orderBy(F.col("date").asc())
# df = df.withColumn("lag1", lag("y", 1).over(partition))
# df = df.withColumn("lag24", lag("y", 24).over(partition)).drop("Address").na.drop(subset=["lag1", "lag24"]).na.fill(0)

# columns = [
#     "date",
#     "Temperature",
#     "Relative Humidity",
#     "Wind Speed",
#     "Visibility",
#     "lag1",
#     "lag24",
#     "y"]

# _df = df.withColumn("filter_col", when(date_format(F.col("date"),"HH:mm:ss").between("07:00:00","23:00:00"),"day").otherwise("night"))

In [ ]:
# # Days
# df_day = _df.filter(F.col("filter_col")=="day").drop("filter_F.col")

# # train test split
# df_day = df_day.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("date")))
# df_day_train = df_day.where("rank <= .8").select(*columns).sort('date')
# df_day_test = df_day.where("rank > .8").select(*columns).sort('date')

# # Chain model, assembler and scaler into a Pipeline.
# assemblers = [
#     VectorAssembler(inputCols=columns[1:-3], outputCol='weather_coditions_vec'),
#     VectorAssembler(inputCols=columns[5:-1], outputCol='autoregressive_features_vec'),
# ]              

# scalers = [
#     MinMaxScaler(inputCol="weather_coditions_vec", outputCol="weather_coditions_vec_scaled"),
#     MinMaxScaler(inputCol="autoregressive_features_vec", outputCol="autoregressive_features_vec_scaled"),
# ]
         
# pipeline = Pipeline(stages=assemblers + scalers)

# scalerModel = pipeline.fit(df_day_train)
# _df_day_train_scaled = scalerModel.transform(df_day_train)
# _df_day_test_scaled = scalerModel.transform(df_day_test)

# df_day_train_scaled = _df_day_train_scaled.select("date", "weather_coditions_vec_scaled", "lag1", "lag24", "y").sort("date")
# df_day_test_scaled = _df_day_test_scaled.select("date", "weather_coditions_vec_scaled", "lag1", "lag24", "y").sort("date")

In [ ]:
# names = {x + "_scaled": x for x in columns_to_scale}
# firstelement=udf(lambda v:float(v[0]),FloatType())

# df_day_train_scaled = _df_day_train_scaled.select([firstelement(c).alias(c) for c in _df_day_test_scaled.columns])
# df_day_train_scaled = df_day_train_scaled.select([F.col(c).alias(names[c]) for c in names.keys()])